# Phone Price Prediction Project

In [1]:
import re
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from itables import show
import seaborn as sns

In [ ]:
df = pd.read_csv('phone_data.csv', index_col='Unnamed: 0')

In [ ]:
show(df, maxBytes=0)

The column names are in Indonesian and they contain spaces. Let's translate them to english and standardize the column format using `column_mapper.csv` before we proceed to data analysis

In [ ]:
column_mapper = pd.read_csv('column_mapper.csv', index_col='col_id').squeeze().to_dict()
df.columns = df.columns.map(column_mapper)
df.columns

# 1. Column Transformation

In [ ]:
show(df, maxBytes=0)

We can see that the scraped datas are not very workable since they are all stored as string. We'll first transform this values into numerical for easier analysis

In [ ]:
clean_df = df.copy()

In [ ]:
## Extract price
clean_df['price'] = clean_df['price'].str.split("Rp").str[1].str.replace(".", "").astype(float)

In [ ]:
## Map Yes or No values to 0 and 1
boolean_cols = [col for col in clean_df.columns if 'has' in col or 'is' in col]
for col in boolean_cols:
    clean_df[col] = clean_df[col].map({'Yes':1, 'No':0})

In [ ]:
## Drop category
clean_df = clean_df.drop(columns=['category'])

In [ ]:
## Extract memory related values
def extract_memory(string):
    if pd.isnull(string):
        return None
    
    if 'GB' in string:
        return float(string.split('GB')[0])
    
    if 'TB' in string:
        return float(string.split('TB')[0]) * 1024

memory_cols = ['ram', 'internal_memory']
for col in memory_cols:
    clean_df[col] = clean_df[col].apply(extract_memory)

In [ ]:
## Extract camera related values
camera_resolution_cols = ['back_camera_resolution', 'front_camera_resolution', 'front_camera_resolution_2']
for col in camera_resolution_cols:
    clean_df[col] = clean_df[col].str.split('MP').str[0].astype(float)

In [ ]:
## Extract values with constant unit
clean_df['weight'] = clean_df['weight'].str.split('g').str[0].astype(float)
clean_df['num_sim_slots'] = clean_df['num_sim_slots'].str.split('slot').str[0].astype(float)
clean_df['screen_size'] = clean_df['screen_size'].str.split('inch').str[0].astype(float)
clean_df['cpu_speed'] = clean_df['cpu_speed'].str.split('GHz').str[0].astype(float)
clean_df['battery_capacity'] = clean_df['battery_capacity'].str.split('mAh').str[0].astype(float)

In [ ]:
## Extract external memory capacity
def extract_memory_capacity(string):
    if pd.isnull(string):
        return None
    
    match = re.search(r'(\d+)\s*(GB|TB)', string, re.IGNORECASE)
    if match:
        value = int(match.group(1))
        return value * 1024 if match.group(2).upper() == 'TB' else value
    return None

clean_df['external_memory_capacity'] = clean_df['external_memory_capacity'].apply(extract_memory_capacity)

### Transforming network column

We'll drop `2g,3g,4g,5g_network` columns since the details are pretty hard to put into the models and instead, we'll just create boolean columns indicating whether a phone has 2g,3g,4g,5g network

In [ ]:
## Transforming network column
for i in range(2,6):
    clean_df[f'has_{i}G_network'] = np.logical_or(f'{i}G' in clean_df['network'], ~clean_df[f'{i}g_network'].isnull())
    clean_df[f'has_{i}G_network'] = clean_df[f'has_{i}G_network'].astype(int)
clean_df = clean_df.drop(columns=['2g_network', '3g_network', '4g_network', '5g_network', 'network'])

### Transforming material column

In [ ]:
def extract_materials(description):
    if pd.isnull(description):
        return ('unspecified', 'unspecified', 'unspecified')
    # Define patterns for each component
    front_pattern = r'(glass|plastic|metal)(?=\s+front)'
    back_pattern = r'(glass|plastic|metal)(?=\s+back)'
    frame_pattern = r'(titanium|aluminum|steel|plastic)(?=\s+frame)'

    # Search for patterns in the description
    front_match = re.search(front_pattern, description, re.IGNORECASE)
    back_match = re.search(back_pattern, description, re.IGNORECASE)
    frame_match = re.search(frame_pattern, description, re.IGNORECASE)

    # Extract the matched materials
    front_material = front_match.group(0) if front_match else 'unspecified'
    back_material = back_match.group(0) if back_match else 'unspecified'
    frame_material = frame_match.group(0) if frame_match else 'unspecified'

    return front_material.lower(), back_material.lower(), frame_material.lower()

In [ ]:
material_df = pd.DataFrame(
    clean_df['material'].apply(extract_materials).to_list(),
    columns=['front_material', 'back_material', 'frame_material'],
    index=clean_df.index
    )
clean_df = pd.concat([clean_df, material_df], axis=1)
clean_df = clean_df.drop(columns=['material'])

### Transforming other main camera

In [ ]:
# Define functions to compute mean, max, and min, handling conversion from strings
def safe_mean(lst):
    if isinstance(lst, list):
        num_lst = [float(i) for i in lst if i.isnumeric()]
        if num_lst:
            return np.mean(num_lst)
    return 0

def safe_max(lst):
    if isinstance(lst, list):
        num_lst = [float(i) for i in lst if i.isnumeric()]
        if num_lst:
            return np.max(num_lst)
    return 0

def safe_min(lst):
    if isinstance(lst, list):
        num_lst = [float(i) for i in lst if i.isnumeric()]
        if num_lst:
            return np.min(num_lst)
    return 0

In [ ]:
other_main_camera = clean_df['other_main_camera_resolution'].str.split('MP').str[0].str.strip().str.split(',')

clean_df['other_main_camera_mean'] = other_main_camera.apply(safe_mean)
clean_df['other_main_camera_max'] = other_main_camera.apply(safe_max)
clean_df['other_main_camera_min'] = other_main_camera.apply(safe_min)

In [ ]:
clean_df = clean_df.drop(columns=['other_main_camera_resolution'])

In [ ]:
show(clean_df, maxBytes=0)

## 2. Handling Missing Values

In [ ]:
def compute_missing_values_pct(df):
    missing_values = pd.DataFrame(columns=['count', 'percentage'])
    missing_values['count'] = df.isnull().astype(int).sum()
    missing_values['percentage'] = missing_values['count'] / df.shape[0]
    missing_values = missing_values.sort_values(by='percentage', ascending=False)

    return missing_values

def show_non_missing(df, col):
    show(df.loc[~df[col].isnull(), [col, 'product_name', 'price']])

In [ ]:
missing_values_pct = compute_missing_values_pct(clean_df)
show(missing_values_pct)

### 1a. Remove missing `price`

In [ ]:
clean_df = clean_df.dropna(subset=['price'])
show(compute_missing_values_pct(clean_df))

### 1b. Remove columns with > 65% missing values

In [ ]:
missing_values_pct = compute_missing_values_pct(clean_df)
missing_values_pct = missing_values_pct[missing_values_pct['percentage'] >= 0.65]
clean_df = clean_df.drop(columns=missing_values_pct.index)

In [ ]:
show(compute_missing_values_pct(clean_df))

### 1c. Fill categorical cols with mode

In [ ]:
boolean_cols = [col for col in clean_df.columns if 'has' in col or 'is' in col]
boolean_fill_value_dict = {
    col:clean_df[col].mode()[0] for col in boolean_cols if col in clean_df.columns
}
clean_df = clean_df.fillna(boolean_fill_value_dict)

In [ ]:
show(compute_missing_values_pct(clean_df))

### 1d. Fill numerical cols with mean

In [ ]:
fill_with_mean_cols = [
    'weight', 'cpu_speed', 'ram', 'back_camera_resolution',
    'front_camera_resolution', 'release_year', 'battery_capacity',
    'screen_size'
]
clean_df = clean_df.fillna(clean_df[fill_with_mean_cols].mean().to_dict())

In [ ]:
show(compute_missing_values_pct(clean_df))

### 1e. Drop categorical cols with too many possible values

In [ ]:
missing_values_pct = compute_missing_values_pct(clean_df).loc[clean_df.dtypes=='object']
missing_values_pct = missing_values_pct[missing_values_pct['percentage'] > 0]

In [ ]:
too_many_unique = clean_df.loc[:, missing_values_pct.index].nunique()
too_many_unique = too_many_unique[too_many_unique > 10].index
too_many_unique

In [ ]:
clean_df = clean_df.drop(columns=too_many_unique)

In [ ]:
show(compute_missing_values_pct(clean_df))

### Handling external memory type

In [ ]:
clean_df = clean_df.fillna({'external_memory_type':clean_df['external_memory_type'].mode()[0]})

### Handling float cols with mean

In [ ]:
show(compute_missing_values_pct(clean_df))

In [ ]:
clean_df['battery_type'].value_counts()

In [ ]:
clean_df.loc[clean_df['battery_type']=='Non-removable', 'battery_type'] = 'Non-Removable'

In [ ]:
clean_df['num_core'].value_counts()

In [ ]:
clean_df['num_core'] = clean_df['num_core'].map({
    'Octa Core':8,
    'Hexa Core':6,
    'Quad Core':4,
    'Dual Core':2,
    'Deca Core':10
})

In [ ]:
clean_df['num_sim_slots'].value_counts()

In [ ]:
clean_df['battery_material'].value_counts()

In [ ]:
clean_df['num_back_camera'].value_counts()

In [ ]:
clean_df['processor'].value_counts()

In [ ]:
clean_df['sim_card_type'].value_counts()

In [ ]:
cols_to_fill = ['battery_type', 'num_core', 'num_sim_slots', 'battery_material', 'num_back_camera', 'processor', 'sim_card_type']
for col in cols_to_fill:
    clean_df.loc[clean_df[col].isnull(), col] = clean_df[col].mode().values[0]

In [ ]:
compute_missing_values_pct(clean_df)

### Screener for personal phone purchase

In [ ]:
screener = clean_df[
    (clean_df['release_year']>=2023) & (clean_df['ram'] >= 8) & (clean_df['internal_memory'] >= 256)
    & (clean_df['has_nfc']==1) & (clean_df['has_5G_network']==1)
    ].sort_values(by='price')
show(screener, maxBytes=0)

## 2. Analyzing `price`

In [ ]:
clean_df['price'].describe()

In [ ]:
sns.displot(clean_df['price'], kde=True)

As always, `price` is positively skewed. Proper way to address this is to use log price later in our model as it is much less skewed, shown in the below plot

In [ ]:
clean_df['log_price'] = np.log(clean_df['price'])
sns.displot(clean_df['log_price'], kde=True)

In [ ]:
clean_df.corr(numeric_only=True)['log_price'].sort_values(ascending=False)